In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import psycopg2
from psycopg2.extras import DictCursor
from science.collector.service.poloniex_public_service import PoloniexPublicService
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
connection = psycopg2.connect("dbname=deep_crypto user=postgres password=postgres host=localhost port=5432")
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

poloniexPublicService = PoloniexPublicService(connection, cursor)

poloniexPublicService.saveChartDataToCSV(main_currency='BTC', secondary_currency='ETH', start='1509483600',
                                         end='1514754000', period='300')

In [ ]:
dataset = read_csv('dataset.csv', index_col=['date'], parse_dates=['date'], dayfirst=True,
                   usecols=['date', 'weighted_average'])

In [ ]:
len(dataset)

In [ ]:
def split_dataset(series, percentage):
    X = series.values
    size = int(len(X) * percentage)
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train]
    return [history, test]

In [ ]:
train, test = split_dataset(dataset, 0.98)

In [ ]:
def evaluate_model(params, dataset, print_period):
    [P, D, Q, s] = params
    [train, test] = dataset
    
    predictions = list()

    for t in range(len(test)):
        model = SARIMAX(train, seasonal_order=(P, D, Q, s))
        model_fit = model.fit(disp=0, maxiter=1000, method='nm')
        
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        
        obs = test[t]
        train.append(obs)
        
        if (t // print_period) == 0: 
            print('predicted=%f, expected=%f' % (yhat, obs))
    
    error = mean_squared_error(test, predictions)
    rmse = sqrt(error)

In [ ]:
error = evaluate_model([5, 1, 0, 2], [train, test], 1)

In [ ]:
print(model_fit.mle_retvals)

error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()